## Adaptive MD

Using __`radical.pilot`__

In [1]:
import os
import sys
print 'python version', sys.version.split()[0]

python version 2.7.9


In [2]:
# set default verbose level
print os.environ.get('RADICAL_PILOT_VERBOSE')
print os.environ.get('RADICAL_PILOT_DBURL')
verbose = os.environ.get('RADICAL_PILOT_VERBOSE', 'REPORT')
os.environ['RADICAL_PILOT_VERBOSE'] = verbose

# set default URL to IMP Mongo DB
# path_to_db = os.environ.get(
#     'RADICAL_PILOT_DBURL', "mongodb://ensembletk.imp.fu-berlin.de:27017/rp")

# assume we run a local
path_to_db = os.environ.get(
    'RADICAL_PILOT_DBURL', "mongodb://localhost:27017/rp")

os.environ['RADICAL_PILOT_DBURL'] = path_to_db

REPORT
mongodb://localhost:27017/rp


In [3]:
import radical.pilot as rp
import radical.utils as ru

/Users/jan-hendrikprinz/anaconda/envs/rp/lib/python2.7/site-packages/radical/utils/atfork/stdlib_fixer.py:58: UserWarning: logging module already imported before fixup.
  warnings.warn('logging module already imported before fixup.')


In [4]:
# import adaptive components

from adaptivemd import OpenMMEngine, AllegroCluster, Brain, MDCluster, \
    LocalCluster, File, NTrajectories
import adaptivemd.util as amp
from adaptivemd import Copy, Move, Link
from adaptivemd import Task
from adaptivemd.task import PythonTask
from adaptivemd.analysis import PyEMMAAnalysis

import numpy as np

#### 1. Select a resource

We pick to run on this machine

In [5]:
resource = LocalCluster(150, 2)

We still need to add the necessary path to find the conda installation

In [6]:
resource.add_path(amp.path_conda_local_jhp)

Set a reporter for radical pilot

In [8]:
# we use a reporter class for nicer output
report = ru.LogReporter(name='radical.pilot', level=verbose)
report.title('Getting Started (RP version %s)' % rp.version)


 Getting Started (RP version v0.44.RC1)                                         



pick a file from the currend working directory

In [9]:
pdb_file = File('file://input.pdb')

create the engine

In [10]:
engine = OpenMMEngine(
    pdb_file=pdb_file,
    system_file=File('file://system.xml'),
    integrator_file=File('file://integrator.xml'))

In [11]:
modeller = PyEMMAAnalysis(
    pdb_file=pdb_file, 
    source_folder=File('../staging_area/alanine/trajs'))

#### 3. Create the cluster used

In [12]:
cluster = MDCluster(
    system='alanine',
    resource=resource,
    report=report)

In [13]:
# add the path to CONDA if now already in the default
cluster.add_path(os.environ.get('CONDA_PATH', '~/anaconda/bin'))

In [14]:
cluster.register(engine)
cluster.register(modeller)

select a strategy aka _brain_

In [15]:
brain = Brain(cluster)  # this needs to be smarter

In [16]:
w = resource.wrapper
w.append('export MODULEPATH=/import/ag_cmb/software/modules/:$MODULEPATH')
w.append('module load cuda/7.5')

So let's do it...

In [18]:
cluster.enter()

new session: [rp.session.Stevie.fritz.box.jan-hendrikprinz.017200.0000]        \
database   : [mongodb://localhost:27017/rp]                                   ok
create pilot manager                                                          ok
create unit manager                                                           ok

--------------------------------------------------------------------------------
submit pilots                                                                   

create pilot description [local.localhost:2]                                  ok
submit 1 pilot(s) .                                                           ok
add 1 pilot(s)                                                                ok


In [40]:
self.folder_name = '%s-%s' % (cluster.session._uid, cluster.pilot._uid)

'rp.session.Stevie.fritz.box.jan-hendrikprinz.017200.0000-pilot.0000'

In [43]:
from os.path import *

In [52]:
relpath('rp.session.Stevie.fritz.box.jan-hendrikprinz.017200.0000-pilot.0000/staging_area/trajs', 'rp.session.Stevie.fritz.box.jan-hendrikprinz.017200.0000-pilot.0001/staging_area/trajs')

'../../../rp.session.Stevie.fritz.box.jan-hendrikprinz.017200.0000-pilot.0000/staging_area/trajs'

In [53]:
f = File('staging:///trajs/output.dcd')

In [57]:
f.location

'staging:///trajs/output.dcd'

In [25]:
t = engine.task_run_trajectory(cluster.new_trajectory(engine['pdb'], 10, number=1))

In [28]:
t.pre_exec

['export MODULEPATH=/import/ag_cmb/software/modules/:$MODULEPATH',
 'module load cuda/7.5']

In [17]:
cluster.submit(cluster.new_trajectory(engine['pdb'], 10, number=1))

submit 1 unit(s)
        .                                                                     ok


In [18]:
def task_generator(cluster):
    return [
        cluster['engine'].task_run_trajectory(traj) for traj in
        cluster.new_ml_trajectory(10, 2)]

In [19]:
cluster\
    .on(cluster.on_ntraj(1))\
    .do(task_generator)

In [20]:
cluster\
    .on(cluster.on_ntraj(1))\
    .do(task_generator)\
    .repeat().until(cluster.on_ntraj(20))

In [39]:
cluster\
    .on(cluster.on_ntraj(range(25, 28)))\
    .do(task_generator)\
#     .repeat().until(cluster.on_ntraj(20))

In [34]:
cluster._events

In [40]:
cluster.trigger()

submit 2 unit(s)
        ..                                                                    ok
submit 2 unit(s)
        ..                                                                    ok


In [44]:
for ev in cluster._events:
    print ev, bool(ev), ev.has_running_tasks, ev._active_tasks

In [45]:
cluster.trigger()

In [46]:
print cluster.trajectories

<ViewBundle with 29 file(s) @ 0x10e86f490>


In [51]:
cluster.exit()


--------------------------------------------------------------------------------
finalize                                                                        

closing session rp.session.Stevie.fritz.box.jan-hendrikprinz.017200.0000       \
close pilot managerO |-\|/-\|/-\|/-\|/-\|/-\|                                                           \
wait for 1 pilot(s) O/*|                                                         ok
                                                                              ok
close unit manager                                                            ok
session lifetime: 878.5s                                                      ok


In [39]:
for m in cluster.models:
    print
    print m['msm']['P']


[[0.7469879482893063, 0.09073996619471751, 0.11995289595797451, 0.0, 0.042319189558001624], [0.09119556620756632, 0.7959183626805835, 0.11288607111185008, 0.0, 0.0], [0.08780494724049683, 0.08221924720109454, 0.7669902893260713, 0.048480500889836645, 0.014505015342500784], [0.0, 0.0, 0.06312169777909123, 0.9099099174080121, 0.02696838481289669], [0.16277189682530221, 0.0, 0.07621703082043221, 0.10883715897805557, 0.65217391337621]]


In [68]:
msm = np.array(cluster.models[-1]['msm']['P'])
print msm

[[ 0.84560906  0.00568267  0.07243243  0.          0.07627584]
 [ 0.00841153  0.88701518  0.          0.02196144  0.08261185]
 [ 0.06567568  0.          0.84811828  0.01989899  0.06630705]
 [ 0.          0.03010874  0.04453631  0.88631091  0.03904404]
 [ 0.05706761  0.04175633  0.05471305  0.01439471  0.83206831]]


In [69]:
len(cluster.models)

4

In [71]:
len(cluster.files)

44

In [73]:
cluster.wait()

report.header('generated new trajectories')

for f in cluster.files:
    report.info(repr(f) + '\n\t')


--------------------------------------------------------------------------------
generated new trajectories                                                      

Trajectory(Trajectory('input.pdb'                               00000005.dcd[0..100])[93]                               00000012.dcd[0..100])
        Trajectory(Trajectory('input.pdb'                       00000004.dcd[0..100])[46]                       00000039.dcd[0..100])
        Trajectory(Trajectory(Trajectory('input.pdb'  00000004.dcd[0..100])[76]  00000017.dcd[0..100])[39]  00000037.dcd[0..100])
        Trajectory(Trajectory('input.pdb'                       00000005.dcd[0..100])[71]                       00000018.dcd[0..100])
        Trajectory(Trajectory('input.pdb'                       00000004.dcd[0..100])[76]                       00000017.dcd[0..100])
        Trajectory(Trajectory(Trajectory('input.pdb'  00000007.dcd[0..100])[5]  00000014.dcd[0..100])[23]  00000030.dcd[0..100])
        Trajectory(Trajectory('in